In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import time

Observe the accuracy of each classifier in more detail. We tested classifiers based on both BoW and Word2vec training data. 


### In the classification report: 

Precision: the ability to find elements correctly in a class. 
Ex: Precision for Positive class = True Positive / ( True Positive + False Positive )

Recall: the ability to find the elements of a class. 
Ex: Recall for Positive class = True Positive / ( True Positive + False Negative )

F1 Score: 2 * ( Recall * Precision ) / ( Recall + Precision )

Support is the number of actual occurrences of the class in the specified dataset.

## BoW data (sparse)

In [3]:
import scipy

X = load = scipy.sparse.load_npz('../new_data/X_sparse.npz')
Y = np.genfromtxt('../new_data/Y.csv', delimiter=',')[1:]

X_comp, X_test, Y_comp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
Xtr, Xva, Ytr, Yva = train_test_split(X_comp, Y_comp, test_size=0.2)

In [4]:
# grid search results (hyper-values): 

lr = LogisticRegression(C=0.1, penalty='l2')

mlp = MLPClassifier(max_iter=1000, hidden_layer_sizes=(200, 50, 50), activation='relu', 
                            solver='lbfgs', alpha=0.0001, learning_rate='constant')

gb = GradientBoostingClassifier(loss="deviance", learning_rate=0.2, 
                                n_estimators=1500, max_depth=3,
                                min_samples_split=6,min_samples_leaf=1,
                                max_features='sqrt', subsample=0.95)

In [5]:
print("lr training started")
lr.fit(Xtr,Ytr)
print("lr training finished")

print("mlp training started")
mlp.fit(Xtr,Ytr)
print("mlp training finished")

print("gb training started")
gb.fit(Xtr,Ytr)
print("gb training finished")

lr training started
lr training finished
mlp training started
mlp training finished
gb training started
gb training finished


In [10]:
from sklearn.ensemble import VotingClassifier

combined = VotingClassifier(estimators=[('lr', lr), ('nn', mlp), ('gb', gb)],
                            voting='soft', flatten_transform=True, weights=[2,1,3])

print("training started")
combined.fit(Xtr,Ytr)
print("training finished")

training started
training finished


### Classification Report of Different Classifiers

In [6]:
target_names = ['negative', 'positive']

In [7]:
lr_pred = lr.predict(X_test)

print("Logistic Regression Classifier Report")
print(classification_report(Y_test, lr_pred, target_names=target_names))

Logistic Regression Classifier Report
              precision    recall  f1-score   support

    negative       0.78      0.72      0.75     10031
    positive       0.74      0.79      0.77      9969

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000



In [8]:
mlp_pred = mlp.predict(X_test)

print("Neural Network Classifier Report")
print(classification_report(Y_test, mlp_pred, target_names=target_names))

Neural Network Classifier Report
              precision    recall  f1-score   support

    negative       0.73      0.71      0.72     10031
    positive       0.71      0.73      0.72      9969

    accuracy                           0.72     20000
   macro avg       0.72      0.72      0.72     20000
weighted avg       0.72      0.72      0.72     20000



In [9]:
gb_pred = gb.predict(X_test)

print("Gradient Boosting Classifier Report")
print(classification_report(Y_test, gb_pred, target_names=target_names))

Gradient Boosting Classifier Report
              precision    recall  f1-score   support

    negative       0.79      0.71      0.75     10031
    positive       0.74      0.80      0.77      9969

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000



In [11]:
comb_pred = combined.predict(X_test)

print("Ensemble Classifier Report")
print(classification_report(Y_test, comb_pred, target_names=target_names))

Ensemble Classifier Report
              precision    recall  f1-score   support

    negative       0.77      0.74      0.76     10031
    positive       0.75      0.78      0.76      9969

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000



### Check the similarity of the results predicted by diffierent classifiers

In [10]:
lr_pred_1 = lr_pred[lr_pred==1]
lr_pred_0 = lr_pred[lr_pred==0]

mlp_pred_1 = mlp_pred[mlp_pred==1]
mlp_pred_0 = mlp_pred[mlp_pred==0]

gb_pred_1 = gb_pred[gb_pred==1]
gb_pred_0 = gb_pred[gb_pred==0]

In [11]:
print("Similarity: ")

print("\tlr  vs. mlp:\t%.4f" % ((lr_pred[lr_pred==mlp_pred].shape[0])/(lr_pred.shape[0])))
print("\tlr  vs. gb: \t%.4f" % ((lr_pred[lr_pred==gb_pred].shape[0])/(lr_pred.shape[0])))
print("\tmlp vs. gb: \t%.4f" % ((mlp_pred[mlp_pred==gb_pred].shape[0])/(mlp_pred.shape[0])))

Similarity: 
	lr  vs. mlp:	0.8075
	lr  vs. gb: 	0.9345
	mlp vs. gb: 	0.7932


## Word2vec Data

In [12]:
X = np.genfromtxt('../../new_data/X_word2vec.csv', delimiter=',')[1:]
Y = np.genfromtxt('../new_data/Y.csv', delimiter=',')[1:]

X_comp, X_test, Y_comp, Y_test, indices_train, indices_test = train_test_split(X, Y, np.arange(100000), test_size=0.2, random_state=0)
Xtr, Xva, Ytr, Yva = train_test_split(X_comp, Y_comp, test_size=0.2)

In [14]:
import joblib

mlp = joblib.load('../new_data/best_mlp.sav')

lr = joblib.load('../new_data/best_lr.sav')

gb = joblib.load('../new_data/best_gb.sav')

combined = joblib.load('../new_data/best_combined.sav')

In [15]:
lr_pred = lr.predict(X_test)

print("Logistic Regression Classifier Report")
print(classification_report(Y_test, lr_pred, target_names=target_names))

Logistic Regression Classifier Report
              precision    recall  f1-score   support

    negative       0.76      0.75      0.76     10031
    positive       0.75      0.76      0.76      9969

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000



In [16]:
mlp_pred = mlp.predict(X_test)

print("Neural Network Classifier Report")
print(classification_report(Y_test, mlp_pred, target_names=target_names))

Neural Network Classifier Report
              precision    recall  f1-score   support

    negative       0.76      0.75      0.76     10031
    positive       0.75      0.76      0.76      9969

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000



In [17]:
gb_pred = gb.predict(X_test)

print("Gradient Boosting Classifier Report")
print(classification_report(Y_test, gb_pred, target_names=target_names))

Gradient Boosting Classifier Report
              precision    recall  f1-score   support

    negative       0.76      0.76      0.76     10031
    positive       0.76      0.76      0.76      9969

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000



In [19]:
comb_pred = combined.predict(X_test)

print("Ensemble Classifier Report")
print(classification_report(Y_test, comb_pred, target_names=target_names))

Ensemble Classifier Report
              precision    recall  f1-score   support

    negative       0.77      0.77      0.77     10031
    positive       0.76      0.76      0.76      9969

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000



In [18]:
print("Similarity: ")

print("\tlr  vs. mlp:\t%.4f" % ((lr_pred[lr_pred==mlp_pred].shape[0])/(lr_pred.shape[0])))
print("\tlr  vs. gb: \t%.4f" % ((lr_pred[lr_pred==gb_pred].shape[0])/(lr_pred.shape[0])))
print("\tmlp vs. gb: \t%.4f" % ((mlp_pred[mlp_pred==gb_pred].shape[0])/(mlp_pred.shape[0])))

Similarity: 
	lr  vs. mlp:	0.9897
	lr  vs. gb: 	0.8917
	mlp vs. gb: 	0.8915
